In [1]:
import time
import h5py
import lmdb
import os
import time
from PIL import Image
import tensorflow as tf
import numpy as np
import csv

In [2]:
train_path = r"C:\Users\Miloszek\Desktop\wave\train"
valid_path = r"C:\Users\Miloszek\Desktop\wave\valid"

# Set the image dimensions
image_size = (67, 193)  # Adjust to your desired image size

# Set batch size
batch_size = 32  # Adjust batch size as needed

# Load the image dataset from the directory without splitting
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_path,
    image_size=image_size,
    batch_size=448,
    shuffle=True,
    seed=27,
    label_mode='binary',
    validation_split=0.0  # Set to 0.0 to avoid splitting the dataset
)

valid_ds = tf.keras.preprocessing.image_dataset_from_directory(
    valid_path,
    image_size=image_size,
    batch_size=112,
    shuffle=True,
    seed=27,
    label_mode='binary',
    validation_split=0.0  # Set to 0.0 to avoid splitting the dataset
)

Found 448000 files belonging to 2 classes.
Found 112000 files belonging to 2 classes.


In [ ]:
#DYSK
# Set the number of smaller datasets to create
num_subsets = 10

# Create directories to save the smaller datasets
train_save_dir = r'C:\Users\Miloszek\Desktop\subsets\train_subsets'
valid_save_dir = r'C:\Users\Miloszek\Desktop\subsets\valid_subsets'
os.makedirs(train_save_dir, exist_ok=True)
os.makedirs(valid_save_dir, exist_ok=True)

# Split and save the train dataset

for i, (images, labels) in enumerate(train_ds):
    if i >= num_subsets:
        break
    
    subset_save_path = os.path.join(train_save_dir, f"train_subset_{i}")
    os.makedirs(subset_save_path, exist_ok=True)
    
    csv_save_path = os.path.join(subset_save_path, "labels.csv")
    
    with open(csv_save_path, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["filename", "label"])
        
        for j, (image, label) in enumerate(zip(images, labels)):
            if j >= 448:
                break
            
            image_save_path = os.path.join(subset_save_path, f"image_{j}.png")
            image = tf.keras.preprocessing.image.array_to_img(image)
            image.save(image_save_path, format="png")
            writer.writerow([image_save_path, int(label)])
            

# Split and save the validation dataset
for i, (images, labels) in enumerate(valid_ds):
    if i >= num_subsets:
        break
    
    subset_save_path = os.path.join(valid_save_dir, f"valid_subset_{i}")
    os.makedirs(subset_save_path, exist_ok=True)
    
    csv_save_path = os.path.join(subset_save_path, "labels.csv")
    
    with open(csv_save_path, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["filename", "label"])
        
        for j, (image, label) in enumerate(zip(images, labels)):
            if j >= 112:
                break
            
            image_save_path = os.path.join(subset_save_path, f"image_{j}.png")
            image = tf.keras.preprocessing.image.array_to_img(image)
            image.save(image_save_path, format="png")
            writer.writerow([image_save_path, int(label)])
            

In [ ]:
#HD5F
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import cv2

data_dir = r"C:\Users\Miloszek\Desktop\wave"

def load_images_labels(data_dir, num_images_per_class):
    images = []
    labels = []
    
    class_dirs = os.listdir(data_dir)

    for class_dir in class_dirs:
        class_dir_path = os.path.join(data_dir, class_dir)
        
        if os.path.isdir(class_dir_path):
            image_files = os.listdir(class_dir_path)
            #selected_files = np.random.choice(image_files, num_images_per_class, replace=False)

            for image_file in image_files:
                image_path = os.path.join(class_dir_path, image_file)

                # read the image using OpenCV
                image = cv2.imread(image_path)
                
                # append the image and the label to the lists
                images.append(image)
                labels.append(1 if class_dir == "ones" else 0)  # assuming the classes are encoded in the folder names


    return np.array(images), np.array(labels)


train_images, train_labels = load_images_labels(os.path.join(data_dir, 'train'),448000)
valid_images, valid_labels = load_images_labels(os.path.join(data_dir, 'valid'),112000)

# shuffle the datasets
train_images, train_labels = shuffle(train_images, train_labels, random_state=42)
valid_images, valid_labels = shuffle(valid_images, valid_labels, random_state=42)


In [ ]:
#TFREC
def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def create_tf_example(image, label):
    feature = {
        'image': _bytes_feature(tf.io.encode_jpeg(image).numpy()),
        'label': _int64_feature(label)
    }

    return tf.train.Example(features=tf.train.Features(feature=feature))

def save_tfrecords(images, labels, filename):
    with tf.io.TFRecordWriter(filename) as writer:
        for i in range(len(images)):
            tf_example = create_tf_example(images[i], labels[i])
            writer.write(tf_example.SerializeToString())
            
save_tfrecords(train_images, train_labels, r'C:\Users\Miloszek\Desktop\_tf\train.tfrecords')
save_tfrecords(valid_images, valid_labels, r'C:\Users\Miloszek\Desktop\_tf\valid.tfrecords')

In [ ]:
#HD5F
with h5py.File(r"C:\Users\Miloszek\Desktop\hdf5\train.h5", 'w') as f:
    f.create_dataset('images', data=train_images)
    f.create_dataset('labels', data=train_labels)

In [ ]:
#HD5F
with h5py.File(r"C:\Users\Miloszek\Desktop\hdf5\valid.h5", 'w') as f:
    f.create_dataset('images', data=valid_images)
    f.create_dataset('labels', data=valid_labels)

In [6]:
import efficientnet.keras as efn
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

# Initialize EfficientNetB0 model
ef = efn.EfficientNetB0(input_shape=(67, 193, 3), weights='imagenet', include_top=False)

# Add your own classification layers on top
x = ef.output
x = GlobalAveragePooling2D()(x)  # Global average pooling to reduce spatial dimensions
x = Dense(128, activation='relu')(x)  # Add a fully connected layer with 128 units
x = Dense(1, activation='sigmoid')(x)  # Output layer with sigmoid activation for binary classification

# Create the final model
model = Model(inputs=ef.input, outputs=x)


model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [5]:
#DYSK
# Path to the directories containing the subsets
train_save_dir = r'C:\Users\Miloszek\Desktop\subsets\train_subsets'
valid_save_dir = r'C:\Users\Miloszek\Desktop\subsets\valid_subsets'
# Train the model on the current subset with validation data
    
# Iterate over the directories containing the training subsets
for i in range(10):
    start_time = time.time()
    subset_path = os.path.join(train_save_dir, f"train_subset_{i}")
    
    # Load the images and labels from the current subset
    images = []
    labels = []
    with open(os.path.join(subset_path, "labels.csv")) as f:
        reader = csv.reader(f)
        next(reader)  # Skip the header row
        for row in reader:
            image_path, label = row
            image = tf.keras.preprocessing.image.load_img(image_path)
            image_array = tf.keras.preprocessing.image.img_to_array(image)
            images.append(image_array)
            labels.append(int(label))
    
    # Convert the images and labels to numpy arrays
    images = np.array(images)
    labels = np.array(labels)
    
    # Load the validation subset
    valid_subset_path = os.path.join(valid_save_dir, f"valid_subset_{i}")
    
    valid_images = []
    valid_labels = []
    with open(os.path.join(valid_subset_path, "labels.csv")) as f:
        reader = csv.reader(f)
        next(reader)  # Skip the header row
        for row in reader:
            image_path, label = row
            valid_image = tf.keras.preprocessing.image.load_img(image_path)
            valid_image_array = tf.keras.preprocessing.image.img_to_array(valid_image)
            valid_images.append(valid_image_array)
            valid_labels.append(int(label))
    
    # Preprocess the validation images and labels as necessary
    
    # Convert the validation images and labels to numpy arrays
    valid_images = np.array(valid_images)
    valid_labels = np.array(valid_labels)
    end_time = time.time()

    
    model.fit(images, labels, batch_size=32, epochs=2, validation_data=(valid_images, valid_labels))
    

    elapsed_time = end_time - start_time
    print(elapsed_time)
    


Epoch 1/2
14/14 [==============================] - 100s 3s/step - loss: 0.7070 - accuracy: 0.5379 - val_loss: 0.6984 - val_accuracy: 0.5179
Epoch 2/2
14/14 [==============================] - 29s 2s/step - loss: 0.5254 - accuracy: 0.7522 - val_loss: 0.7349 - val_accuracy: 0.5357
1.1902508735656738
Epoch 1/2
14/14 [==============================] - 32s 2s/step - loss: 0.6739 - accuracy: 0.5938 - val_loss: 0.7020 - val_accuracy: 0.5446
Epoch 2/2
14/14 [==============================] - 33s 2s/step - loss: 0.5140 - accuracy: 0.7478 - val_loss: 0.7391 - val_accuracy: 0.5893
0.760113000869751
Epoch 1/2
14/14 [==============================] - 30s 2s/step - loss: 0.6584 - accuracy: 0.6228 - val_loss: 0.6855 - val_accuracy: 0.4821
Epoch 2/2
14/14 [==============================] - 29s 2s/step - loss: 0.5104 - accuracy: 0.7790 - val_loss: 0.5641 - val_accuracy: 0.7411
0.8656671047210693
Epoch 1/2
14/14 [==============================] - 28s 2s/step - loss: 0.6266 - accuracy: 0.6473 - val_loss: 

In [3]:
#HD5F
def load_slices(h5_file, num_slices):
    with h5py.File(h5_file, 'r') as f:
        images = f['images']
        labels = f['labels']

        slice_size = len(labels) // num_slices

        for i in range(num_slices):
            start_index = i * slice_size
            end_index = (i + 1) * slice_size if i != num_slices - 1 else None  # get all remaining samples in the last slice

            yield np.array(images[start_index:end_index]), np.array(labels[start_index:end_index])

# load each slice and fit it into the model
for i in range(10):
    start_time = time.time()
    # load validation data slice
    train_images_slice, train_labels_slice = next(load_slices(r"C:\Users\Miloszek\Desktop\hdf5\train.h5", 100))
    valid_images_slice, valid_labels_slice = next(load_slices(r"C:\Users\Miloszek\Desktop\hdf5\valid.h5", 100))
    end_time = time.time()
    
    model.fit(train_images_slice, train_labels_slice, batch_size=32, epochs=2, validation_data=(valid_images_slice, valid_labels_slice))
   

    elapsed_time = end_time - start_time
    print(elapsed_time)
    

Epoch 1/2
3/3 [==============================] - 36s 4s/step - loss: 0.7420 - accuracy: 0.4494 - val_loss: 1.1201 - val_accuracy: 0.4545
Epoch 2/2
3/3 [==============================] - 5s 2s/step - loss: 0.3618 - accuracy: 0.8764 - val_loss: 3.6455 - val_accuracy: 0.6364
0.019224882125854492
Epoch 1/2
3/3 [==============================] - 5s 2s/step - loss: 0.1326 - accuracy: 0.9663 - val_loss: 1.2721 - val_accuracy: 0.5909
Epoch 2/2
3/3 [==============================] - 5s 2s/step - loss: 0.0913 - accuracy: 0.9663 - val_loss: 1.0020 - val_accuracy: 0.5909
0.00800180435180664
Epoch 1/2
3/3 [==============================] - 5s 2s/step - loss: 0.0115 - accuracy: 1.0000 - val_loss: 0.8639 - val_accuracy: 0.5909
Epoch 2/2
3/3 [==============================] - 5s 2s/step - loss: 0.0950 - accuracy: 0.9775 - val_loss: 1.1307 - val_accuracy: 0.5909
0.008919000625610352
Epoch 1/2
3/3 [==============================] - 5s 2s/step - loss: 0.0193 - accuracy: 0.9888 - val_loss: 1.4462 - val_ac

In [7]:
#TFREC
def parse_tf_example(example_proto):
    feature_description = {
        'image': tf.io.FixedLenFeature([], tf.string),
        'label': tf.io.FixedLenFeature([], tf.int64),
    }
    example = tf.io.parse_single_example(example_proto, feature_description)
    image = tf.io.decode_jpeg(example['image'])
    label = example['label']
    return image, label

def load_tfrecord_dataset(file_path):
    raw_dataset = tf.data.TFRecordDataset(file_path)
    parsed_dataset = raw_dataset.map(parse_tf_example)
    return parsed_dataset

def load_slices(tfrecord_path, num_slices):
    dataset = load_tfrecord_dataset(tfrecord_path)
    
    slice_images = []
    slice_labels = []

    for image, label in dataset:
        slice_images.append(image)
        slice_labels.append(label)

        if len(slice_images) >= num_slices:
            yield np.array(slice_images), np.array(slice_labels)
            slice_images = []
            slice_labels = []

    if slice_images:
        yield np.array(slice_images), np.array(slice_labels)
        
for i in range(10):
    start_time = time.time()
    train_images_slice, train_labels_slice = next(load_slices(r"C:\Users\Miloszek\Desktop\_tf\train.tfrecords", 448))
    valid_images_slice, valid_labels_slice = next(load_slices(r"C:\Users\Miloszek\Desktop\_tf\valid.tfrecords", 112))
    end_time = time.time()
    
    model.fit(train_images_slice, train_labels_slice, batch_size=32, epochs=2, validation_data=(valid_images_slice, valid_labels_slice))
    

    elapsed_time = end_time - start_time
    print(elapsed_time)

Epoch 1/2
14/14 [==============================] - 78s 2s/step - loss: 0.7331 - accuracy: 0.5290 - val_loss: 0.7698 - val_accuracy: 0.4643
Epoch 2/2
14/14 [==============================] - 25s 2s/step - loss: 0.5331 - accuracy: 0.7232 - val_loss: 0.6965 - val_accuracy: 0.5089
0.6376180648803711
Epoch 1/2
14/14 [==============================] - 25s 2s/step - loss: 0.2777 - accuracy: 0.8705 - val_loss: 0.9732 - val_accuracy: 0.5357
Epoch 2/2
14/14 [==============================] - 25s 2s/step - loss: 0.2492 - accuracy: 0.9040 - val_loss: 0.7635 - val_accuracy: 0.5536
0.21647381782531738
Epoch 1/2
14/14 [==============================] - 25s 2s/step - loss: 0.2019 - accuracy: 0.9152 - val_loss: 0.8597 - val_accuracy: 0.5089
Epoch 2/2
14/14 [==============================] - 25s 2s/step - loss: 0.1253 - accuracy: 0.9509 - val_loss: 0.9908 - val_accuracy: 0.5982
0.23324322700500488
Epoch 1/2
14/14 [==============================] - 25s 2s/step - loss: 0.0484 - accuracy: 0.9933 - val_loss

In [ ]:
#img = Image.open('.png')

# Preprocess the image
#img_array = np.array(img)
#img_array = np.expand_dims(img_array, axis=0)

# Make a prediction with the model
#print(model.predict(img_array))
#prediction = np.argmax(model.predict(img_array))
#print('Dusty' if prediction == 0 else 'Clean')